In [1]:
from src.utils import *

0. Load EFFIS data

In [2]:
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'

# Create output folder structure
outfolder =  os.path.join(project_f, 'output')
os.makedirs(outfolder, exist_ok=True)

outFIREID_folder =  os.path.join(project_f, 'output', 'FIRE_ID')
os.makedirs(outFIREID_folder, exist_ok=True)

outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')
os.makedirs(outFIREYEARDOY_folder, exist_ok=True)

In [3]:
# load wildfires data
effis_path =  os.path.join(project_f, 'input', 'effis_rawdata_20240402', 'modis.ba.poly.shp')
effis = gpd.read_file(effis_path)

# read effis id as an integer
effis['id'] = effis['id'].astype(int)

# Convert the column containing dates to datetime format
effis['date'] = pd.to_datetime(effis['FIREDATE'])

# Change the CRS to EPSG:3035
effis = effis.to_crs(epsg=3035)

# Verify the new CRS
print("New CRS:", effis.crs)

New CRS: EPSG:3035


In [4]:
effis.head()

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,Year,Month,geometry,date
0,27361,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Foggia,San Giovanni Rotondo,95,6.24999999999349,1.0416666666655816,0,...,92.70833333323677,0,0,0,99.99999999999999,FireSeason,2000,1,"POLYGON ((4797962.987 2087017.112, 4797952.991...",2000-01-01
1,27785,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,0,0,0,...,0,20.22471910110087,0,0,100,FireSeason,2000,1,"POLYGON ((2831094.659 2172187.756, 2831272.314...",2000-01-01
2,27809,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Ragusa,Ragusa,126,0,0,0,...,0,90.47619047611867,9.523809523801965,0,0,FireSeason,2000,1,"POLYGON ((4742602.892 1550648.162, 4742420.047...",2000-01-01
3,27827,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,0,0,0,...,80.42452830186782,0.9433962264148719,0,0,0,FireSeason,2000,1,"POLYGON ((2804985.620 2188269.879, 2805524.280...",2000-01-01
4,27865,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,0,0,0,...,76.47058823518165,4.411764705875864,0,0,0,FireSeason,2000,1,"POLYGON ((2821671.755 2110909.984, 2821308.095...",2000-01-01


In [5]:
# create a lighter subset with target columns
effis_sel = effis[['id', 'COUNTRY',
 'FIREDATE','Year',
 'Month','date','AREA_HA','geometry']]

In [6]:
#create additional fields to include DOY
effis_sel['doy'] = effis_sel['date'].dt.dayofyear
year2dig = (effis_sel.Year.astype(str).str[-2:])
year000 = year2dig+str('000')
effis_sel['yeardoy'] =year000.astype(int)+ effis_sel['doy'] 
effis_sel.tail()

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,id,COUNTRY,FIREDATE,Year,Month,date,AREA_HA,geometry,doy,yeardoy
71139,227829,UA,2024-03-09 09:17:00,2024,3,2024-03-09 09:17:00,39,"POLYGON ((6339476.335 3338727.813, 6339510.616...",69,24069
71140,227830,UA,2024-03-25 00:00:00,2024,3,2024-03-25 00:00:00,64,"POLYGON ((6338121.441 3339682.137, 6338094.506...",85,24085
71141,227831,UA,2024-03-25 00:00:00,2024,3,2024-03-25 00:00:00,37,"POLYGON ((6335933.143 3341380.922, 6335989.462...",85,24085
71142,227832,UA,2024-02-28 10:45:00,2024,2,2024-02-28 10:45:00,50,"POLYGON ((6336385.131 3332524.688, 6336388.173...",59,24059
71143,227833,UA,2024-03-31 10:45:00,2024,3,2024-03-31 10:45:00,1041,"POLYGON ((6253456.357 3367500.991, 6253323.425...",91,24091


1. Rasterize FIREID for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    print(f'year {yr} - nr of fires {effis_yr.id.max()}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIREID_{yr}.tif'
    outfile = os.path.join(outFIREID_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'id',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )

2. Rasterize FIRE YEARDOY for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    print(f'year {yr} - nr of fires {effis_yr.id.max()}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIRE_YEARDOY_{yr}.tif'
    outfile = os.path.join(outFIREYEARDOY_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'yeardoy',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )